In [1]:
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain_openai import ChatOpenAI

from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.prompts import PromptTemplate

In [2]:
import pandas as pd
from langchain_openai import OpenAI

df = pd.read_parquet('./data/parquet/sources-camel-kafka.parquet', engine='pyarrow')
df.head()

,project,path,language,tags,description,source
0,camel,/Users/opiske/code/java/camel/components/camel...,java,,,/*\n * Licensed to the Apache Software Foundat...
1,camel,/Users/opiske/code/java/camel/components/camel...,java,,,/*\n * Licensed to the Apache Software Foundat...
2,camel,/Users/opiske/code/java/camel/components/camel...,java,,,/*\n * Licensed to the Apache Software Foundat...
3,camel,/Users/opiske/code/java/camel/components/camel...,java,,,/*\n * Licensed to the Apache Software Foundat...
4,camel,/Users/opiske/code/java/camel/components/camel...,java,,,/*\n * Licensed to the Apache Software Foundat...


In [3]:
from langchain_community.document_loaders import DataFrameLoader
loader = DataFrameLoader(df, page_content_column="source")
docs = loader.load()
print("Num docs: ", len(docs))

Num docs:  12


In [71]:
#
# Retriever setup
#
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

# Retrieve and generate using the relevant snippets of the sources found in the parquet files.
retriever = vectorstore.as_retriever()
prompt = PromptTemplate.from_template("""
    You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. 
    If you don't know the answer, just say that you don't know.     

    Question: {question} 

    Context: {context} 

    Answer:
""")
print(type(retriever))

<class 'langchain_core.vectorstores.VectorStoreRetriever'>


In [83]:
from langchain.schema import BaseRetriever
from langchain_core.vectorstores import VectorStore, VectorStoreRetriever
from langchain.schema import Document

# for the test, we flip the source and metadata lookups
class DelegatingRetriever(VectorStoreRetriever):

    vectorstore: VectorStoreRetriever = None
    
    class Config:
        """Configuration for this pydantic object."""
        arbitrary_types_allowed = True
        
    def _get_relevant_documents(
        self, query: str, *, run_manager: CallbackManagerForRetrieverRun
    ) -> List[Document]:        
        res = retriever.get_relevant_documents(query=query, run_manager=run_manager) 
        print(res[0])
        return res

    async def _aget_relevant_documents(
        self, query: str, *, run_manager: AsyncCallbackManagerForRetrieverRun
    ) -> List[Document]:        
        pass

In [84]:
# let's take a look at how the retriever actually works
delegating_retriever = DelegatingRetriever()
matches = delegating_retriever.get_relevant_documents("How can I send events to a Kafka broker?")
[print(m.page_content) for m in matches]

page_content='String uri = "kafka:mytopic?brokers=dev1:12345,dev2:12566";\n\n        KafkaEndpoint endpoint = (KafkaEndpoint) context.getComponent("kafka").createEndpoint(uri, params);' metadata={'description': '', 'language': 'java', 'path': '/Users/opiske/code/java/camel/components/camel-kafka/src/test/java/org/apache/camel/component/kafka/KafkaComponentTest.java', 'project': 'camel', 'tags': ''}
String uri = "kafka:mytopic?brokers=dev1:12345,dev2:12566";

        KafkaEndpoint endpoint = (KafkaEndpoint) context.getComponent("kafka").createEndpoint(uri, params);
String uri = "kafka:mytopic?brokers=dev1:12345,dev2:12566";

        KafkaEndpoint endpoint = (KafkaEndpoint) context.getComponent("kafka").createEndpoint(uri, params);
endpoint = kafka.createEndpoint("kafka:sometopic", "sometopic", new HashMap());
        endpoint.doBuild();
        assertTrue(endpoint.getKafkaClientFactory() instanceof DefaultKafkaClientFactory);

        producer = new KafkaProducer(endpoint);

        fro

[None, None, None, None]

In [5]:
#
# The actual Q&A involving the LLM
#

llm = ChatOpenAI(model_name="gpt-3.5-turbo-1106", temperature=0)

def format_docs(docs):    
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

#result = rag_chain.invoke("Show me a Java example of processAsyncSendsMessageWithException in Kafka")
#print(result)

questions = [
    "How can I process exception during an async send in Kafka? Can you show me an example in Java?",    
    "How can I send events to a Kafka broker? Can you show me an full camel route example in Java?",
    "How can specify the partition key when sending events to a Kafka broker? Can you show me an full camel route example in Java?"
]

from langchain.globals import set_debug
set_debug(False)

for i,q in enumerate(questions):
    print(f"Processing question {i} ...")
    result = rag_chain.invoke(q)
    print(f"{i}: {result} \n\n")



Processing question 0 ...
0: To process an exception during an async send in Kafka in Java, you can use the following example:

```java
@Test
public void processAsyncSendsMessageWithException() {
    endpoint.getConfiguration().setTopic("sometopic");
    Mockito.when(exchange.getIn()).thenReturn(in);
    Mockito.when(exchange.getMessage()).thenReturn(in);

    // set up the exception here
    org.apache.kafka.clients.producer.Producer kp = producer.getKafkaProducer();
    Mockito.when(kp.send(any(ProducerRecord.class), any(Callback.class))).thenThrow(new ApiException());

    in.setHeader(KafkaConstants.PARTITION_KEY, 4);

    producer.process(exchange, callback);
    Mockito.verify(producer.getKafkaProducer()).send(any(ProducerRecord.class));
    assertRecordMetadataExists();
}
```

This example sets up the exception using Mockito to throw an `ApiException` during the async send process in Kafka. It then verifies that the producer sends the message and asserts that the record metadata